<h1>Decorator Basics<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Basic-decorator" data-toc-modified-id="Basic-decorator-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Basic decorator</a></span></li><li><span><a href="#Support-Input-Arguments-for-Decorated-Function" data-toc-modified-id="Support-Input-Arguments-for-Decorated-Function-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Support Input Arguments for Decorated Function</a></span></li><li><span><a href="#Handle-Returns-of-The-Decorated-Functions" data-toc-modified-id="Handle-Returns-of-The-Decorated-Functions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Handle Returns of The Decorated Functions</a></span></li><li><span><a href="#Wrap-the-Decorated-Function-to-Preserve-Metadata" data-toc-modified-id="Wrap-the-Decorated-Function-to-Preserve-Metadata-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Wrap the Decorated Function to Preserve Metadata</a></span></li><li><span><a href="#Define-Decorators-With-Arguments" data-toc-modified-id="Define-Decorators-With-Arguments-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Define Decorators With Arguments</a></span></li><li><span><a href="#Class-based-Decorators" data-toc-modified-id="Class-based-Decorators-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Class-based Decorators</a></span></li></ul></div>

In [1]:
import sys
import time

In [2]:
print(sys.executable)
print(sys.version)

C:\Users\r2d4\miniconda3\envs\py3\python.exe
3.8.3 (default, May 19 2020, 06:50:17) [MSC v.1916 64 bit (AMD64)]


## Basic decorator

- In essence, the decorator is a function that accepts another function as its input argument.
- It defines an inner function that actually provides decoration activities and returns the inner function as the output. 
- To use the decorator, you simply place the decorator function name with an @ sign prefix above the function that you want to decorate.

Step by step:
- Firstly, we define the actual decorator. It accepts a single argument, which is the function we are trying to decorate. It will return the "wrapper".
- Inside, we define a wrapper function that _is returned_ and used in place of the original decorated function.

In [3]:
# Basic time logging decorator, for a function that does not take any arguments

def logging_time(func):
    """Decorator that logs time."""
    def logger():
        """Function that logs time."""
        start = time.time()
        func()
        print(f"Calling {func.__name__}: {time.time() - start:.5f}")
    return logger

In [4]:
@logging_time
def calculate_sum():
    return sum(range(1000000))

In [5]:
calculate_sum()

Calling calculate_sum: 0.06100


## Support Input Arguments for Decorated Function

There is a problem with the code snippet above: It assumes that the decorated functions don’t require any input arguments.
To address this issue, we should consider using * args and ** kwargs with the _inner_ decorator definition.     


In [6]:
# Revised version capable of handling input arguments

def logging_time(func):
    """Decorator that logs time."""
    def logger(*args, **kwargs):
        """Function that logs time."""
        start = time.time()
        func(*args, **kwargs)
        print(f"Calling {func.__name__}: {time.time() - start:.5f}")

    return logger

In [7]:
@logging_time
def calculate_sum_n(n):
    return sum(range(n))

@logging_time
def say_hi(whom, greeting="Hello"):
    """Greet someone"""
    print(f"{greeting}, {whom}!")

In [8]:
calculate_sum_n(100000)

Calling calculate_sum_n: 0.00400


In [9]:
say_hi("raph", "hi")

hi, raph!
Calling say_hi: 0.00000


## Handle Returns of The Decorated Functions

You can see we store the returned value in result on line 4. But before returning it, we have to finish timing the function. This is an example of behaviour that would not be possible without decorators.

In [10]:
# Slightly different logging example

def timer_decorator(func):
    def timer_wrapper(*args, **kwargs):
        import datetime as dt
        before = dt.datetime.now()                     
        result = func(*args,**kwargs)                
        after = dt.datetime.now()                      
        print("Elapsed Time = {after-before}")
        return result
    
    return timer_wrapper

In [11]:
@timer_decorator
def sum_function(x, y):
    print(x + y)
    return x + y

In [12]:
result = sum_function(2, 5)
print(f"\nResult: {result}")

7
Elapsed Time = {after-before}

Result: 7


## Wrap the Decorated Function to Preserve Metadata

The decoration will by default mess up the metadata of the decorated function, such as docstrings. To solve this problem, we can use another decorator function (wraps) that is shipped in the standard Python library, as shown below:

In [13]:
# The decorated function now has the wrong docstrings.
print(say_hi.__doc__)

Function that logs time.


In [14]:
# Revised decorator using the imported wraps() decorator

from functools import wraps

def logging_time(func):
    """Decorator that logs time."""
    @wraps(func)
    def logger(*args, **kwargs):
        """Function that logs time."""
        start = time.time()
        func(*args, **kwargs)
        print(f"Calling {func.__name__}: {time.time() - start:.5f}")

    return logger

In [15]:
@logging_time
def say_hi(whom, greeting="Hello"):
    "Greet someone."
    print(f"{greeting}, {whom}!")

In [16]:
# The decorated function now has the correct docstrings.
print(say_hi.__doc__)

Greet someone.


## Define Decorators With Arguments

The reason for adding another layer to get the decorator to accept arguments is that the decoration process is chaining the function call. Calling logging_time("ms") will allow us to get the logger function, which has exactly the same function signature as the decorator function that we defined earlier.

Please note that the current definition of the decorators _requires_ that we specify the unit for the decoration. If you want to make your arguments optional, it needs extra work. (see [here](https://medium.com/better-programming/how-to-write-python-decorators-that-take-parameters-b5a07d7fe393) for more info).

In [17]:
# Refined decorator to display the time in the unit that’s specified by the user
# (either in milliseconds or seconds).

def logging_time(unit):
    """Decorator that logs time."""
    def logger(func):
        @wraps(func)
        def inner_logger(*args, **kwargs):
            """Function that logs time."""
            start = time.time()
            func(*args, **kwargs)
            scaling = 1000 if unit == "ms" else 1
            print(f"Calling {func.__name__}: {(time.time() - start) * scaling:.5f} {unit}")

        return inner_logger

    return logger

In [18]:
@logging_time("ms")
def calculate_sum_ms(n):
    """Calculate sum of 0 to n-1"""
    return sum(range(n))

@logging_time("s")
def calculate_sum_s(n):
    """Calculate sum of 0 to n-1"""
    return sum(range(n))

In [19]:
calculate_sum_ms(10000000)

Calling calculate_sum_ms: 528.99837 ms


In [20]:
calculate_sum_s(10000000)

Calling calculate_sum_s: 0.42451 s


## Class-based Decorators

It is possible to decorate using classes instead of functions. The only difference is the syntax, so do what you are more comfortable with.

In [21]:
# Example of a logging decorator class

class Logging: 
  
    def __init__(self, function): 
        self.function = function 
  
    def __call__(self, *args, **kwargs):
      print(f'Before {self.function.__name__}')
      self.function(*args, **kwargs)
      print(f'After {self.function.__name__}')

In [22]:
@Logging
def sum(x, y):
  print(x + y)

In [23]:
sum(3, 7)

Before sum
10
After sum


In [24]:
# Example of a repeater decorator class (accecpting an argument)

class Repeat:
    def __init__(self, n):
        self.n = n

    def __call__(self, func):
        def repeater(*args, **kwargs):
            for _ in range(self.n):
                func(*args, **kwargs)

        return repeater
    

In [25]:
@Repeat(n=2)
def morning_greet(person):
    print(f"Good Morning, {person}!")

In [26]:
morning_greet("raph")

Good Morning, raph!
Good Morning, raph!


---